<a href="https://colab.research.google.com/github/HiroshiATakano/GitHub/blob/master/5%E7%AB%A0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import os

path = os.getcwd()

print(path)

try:
    os.chdir(path + "/drive/My Drive/実践データ分析_100本ノック")
except:
    pass

os.getcwd()

/content


'/content/drive/My Drive/実践データ分析_100本ノック'

In [3]:
!pip install japanize-matplotlib

     |████████████████████████████████| 4.1 MB 5.1 MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120274 sha256=f3bd8829f3c9a67cbebaa2913c17b31c82297c3f2fff6f5990ea8a0d8a958378
  Stored in directory: /root/.cache/pip/wheels/83/97/6b/e9e0cde099cc40f972b8dd23367308f7705ae06cd6d4714658
Successfully built japanize-matplotlib


In [4]:
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import japanize_matplotlib

%matplotlib inline

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactiveity = "all"

In [6]:
import glob

for f in glob.glob('./Data/5章/*.csv'):
  print(os.path.split(f)[1])

customer_join.csv
use_log_months.csv


In [7]:
import pathlib

d = {}

p_temp = pathlib.Path('./Data/5章').glob('*.csv')

for p in p_temp:
  print(p)
  f = p.name.split('.')[0] 
  df = pd.read_csv('./Data/5章/' + p.name)
  d[f]= df

Data/5章/customer_join.csv
Data/5章/use_log_months.csv


#ノック４１：データ読み込み、整形

In [8]:
customer = d['customer_join']
uselog_months = d['use_log_months']

In [9]:
year_months = list(uselog_months["年月"].unique())
year_months

[201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 201901,
 201902,
 201903]

In [10]:
uselog = pd.DataFrame()
for i in range(1,len(year_months)):
  tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]].copy()
  tmp.rename(columns={"count":"count_0"},inplace=True)
  tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]].copy()
  del tmp_before["年月"]
  tmp_before.rename(columns={"count":"count_1"},inplace=True)
  tmp = pd.merge(tmp,tmp_before,on="customer_id",how="left")
  uselog = pd.concat([uselog,tmp],ignore_index=True)
uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


#ノック４２：退会前月の大会顧客データ作成

In [11]:
from dateutil.relativedelta import relativedelta

In [12]:
exit_customer = customer.loc[customer["is_deleted"]==1].copy()
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
exit_customer["exit_date"] = exit_customer['end_date'].map(lambda x: x - relativedelta(months=1))
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
exit_customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,年月
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30,201803
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,デイタイム,7500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30,201803
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,1.0,1.0,1,1,0,2018-04-30,23,2018-03-30,201803
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,通常,3.5,3.5,4,3,1,2018-05-31,23,2018-04-30,201804
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-05-31,23,2018-04-30,201804


In [13]:
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog,exit_customer,on=["customer_id","年月"],how="left")
exit_uselog = exit_uselog.dropna(subset=["name"])
exit_uselog

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,1.0,ナイト,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30758,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31,CA1,1.0,ナイト,6000.0,通常,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28
30787,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31,CA3,1.0,ナイト,6000.0,入会費無料,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28
30827,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31,CA3,1.0,デイタイム,7500.0,入会費無料,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28
30842,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31,CA1,1.0,デイタイム,7500.0,通常,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28


#ノック４３：継続顧客データ

In [29]:
conti_customer = customer.loc[customer["is_deleted"]==0].copy()
conti_uselog = pd.merge(uselog,conti_customer, on="customer_id", how='left')
conti_uselog = conti_uselog.dropna(subset=["name"])
conti_uselog.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27422 entries, 0 to 33850
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   年月                 27422 non-null  object 
 1   customer_id        27422 non-null  object 
 2   count_0            27422 non-null  int64  
 3   count_1            26734 non-null  float64
 4   name               27422 non-null  object 
 5   class              27422 non-null  object 
 6   gender             27422 non-null  object 
 7   start_date         27422 non-null  object 
 8   end_date           0 non-null      object 
 9   campaign_id        27422 non-null  object 
 10  is_deleted         27422 non-null  float64
 11  class_name         27422 non-null  object 
 12  price              27422 non-null  float64
 13  campaign_name      27422 non-null  object 
 14  mean               27422 non-null  float64
 15  median             27422 non-null  float64
 16  max                274

In [30]:
#conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id", keep='last').reset_index(drop=True)
conti_uselog

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201903,AS002855,4,5.0,XXXX,C03,F,2016-11-01,NaN,CA1,0.0,ナイト,6000.0,通常,4.500000,5.0,7.0,2.0,1.0,2019-04-30,29.0
1,201903,AS009373,5,6.0,XX,C01,F,2015-11-01,NaN,CA1,0.0,オールタイム,10500.0,通常,5.083333,5.0,7.0,3.0,1.0,2019-04-30,41.0
2,201903,AS015233,8,4.0,XXXXX,C01,M,2018-05-13,NaN,CA2,0.0,オールタイム,10500.0,入会費半額,7.545455,7.0,11.0,4.0,1.0,2019-04-30,11.0
3,201903,AS015315,4,5.0,XXXXX,C01,M,2015-07-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.833333,5.0,7.0,3.0,1.0,2019-04-30,45.0
4,201903,AS015739,6,6.0,XXXXX,C03,M,2017-06-01,NaN,CA1,0.0,ナイト,6000.0,通常,5.583333,5.5,8.0,4.0,1.0,2019-04-30,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2837,201903,TS995853,8,11.0,XXXX,C01,M,2019-02-08,NaN,CA1,0.0,オールタイム,10500.0,通常,9.500000,9.5,11.0,8.0,1.0,2019-04-30,2.0
2838,201903,TS998593,8,7.0,XXXXX,C03,M,2018-09-01,NaN,CA1,0.0,ナイト,6000.0,通常,8.142857,8.0,9.0,7.0,1.0,2019-04-30,7.0
2839,201903,TS999079,3,2.0,XXX,C03,M,2016-06-01,NaN,CA1,0.0,ナイト,6000.0,通常,4.916667,5.5,9.0,2.0,1.0,2019-04-30,34.0
2840,201903,TS999231,6,6.0,XXXX,C01,M,2017-03-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.666667,5.0,8.0,1.0,1.0,2019-04-30,25.0


In [31]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
predict_data

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201903,AS002855,4,5.0,XXXX,C03,F,2016-11-01,NaN,CA1,0.0,ナイト,6000.0,通常,4.500000,5.0,7.0,2.0,1.0,2019-04-30,29.0,NaT
1,201903,AS009373,5,6.0,XX,C01,F,2015-11-01,NaN,CA1,0.0,オールタイム,10500.0,通常,5.083333,5.0,7.0,3.0,1.0,2019-04-30,41.0,NaT
2,201903,AS015233,8,4.0,XXXXX,C01,M,2018-05-13,NaN,CA2,0.0,オールタイム,10500.0,入会費半額,7.545455,7.0,11.0,4.0,1.0,2019-04-30,11.0,NaT
3,201903,AS015315,4,5.0,XXXXX,C01,M,2015-07-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.833333,5.0,7.0,3.0,1.0,2019-04-30,45.0,NaT
4,201903,AS015739,6,6.0,XXXXX,C03,M,2017-06-01,NaN,CA1,0.0,ナイト,6000.0,通常,5.583333,5.5,8.0,4.0,1.0,2019-04-30,22.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31 00:00:00,CA1,1.0,ナイト,6000.0,通常,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28
3942,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31 00:00:00,CA3,1.0,ナイト,6000.0,入会費無料,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28
3943,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31 00:00:00,CA3,1.0,デイタイム,7500.0,入会費無料,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28
3944,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31 00:00:00,CA1,1.0,デイタイム,7500.0,通常,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28


#ノック４４：予測する月の在籍期間

In [36]:
predict_data["now_date"] = pd.to_datetime(predict_data["年月"],format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3946 entries, 0 to 3945
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   年月                 3946 non-null   object        
 1   customer_id        3946 non-null   object        
 2   count_0            3946 non-null   int64         
 3   count_1            3829 non-null   float64       
 4   name               3946 non-null   object        
 5   class              3946 non-null   object        
 6   gender             3946 non-null   object        
 7   start_date         3946 non-null   datetime64[ns]
 8   end_date           1104 non-null   object        
 9   campaign_id        3946 non-null   object        
 10  is_deleted         3946 non-null   float64       
 11  class_name         3946 non-null   object        
 12  price              3946 non-null   float64       
 13  campaign_name      3946 non-null   object        
 14  mean    

In [37]:
def delta(a,b):
  d = relativedelta(a,b)
  return d.years*12 + d.months

predict_data["period"] = predict_data.apply(lambda x: delta(x['now_date'],x['start_date']),axis=1)
predict_data.sample(n=5)

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,now_date,period
1754,201903,OA101976,6,5.0,XXXXX,C03,M,2016-01-01,NaN,CA1,0.0,ナイト,6000.0,通常,4.666667,5.0,6.0,3.0,1.0,2019-04-30,39.0,NaT,2019-03-01,38
1856,201903,OA374469,7,9.0,XXXXX,C01,F,2019-02-12,NaN,CA1,0.0,オールタイム,10500.0,通常,8.000000,8.0,9.0,7.0,0.0,2019-04-30,2.0,NaT,2019-03-01,0
1399,201903,IK039344,8,6.0,XXX,C03,M,2018-12-15,NaN,CA3,0.0,ナイト,6000.0,入会費無料,7.250000,7.0,9.0,6.0,1.0,2019-04-30,4.0,NaT,2019-03-01,2
1339,201903,HI893036,6,4.0,XXXX,C01,M,2017-10-01,NaN,CA1,0.0,オールタイム,10500.0,通常,6.500000,6.5,11.0,4.0,1.0,2019-04-30,18.0,NaT,2019-03-01,17
2796,201903,TS913591,3,4.0,XXXXXX,C01,F,2016-08-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.583333,4.0,7.0,3.0,1.0,2019-04-30,32.0,NaT,2019-03-01,31
